In [29]:
%pip install tensorflow
%pip install pandas
%pip install scikit-learn
%pip install imbalanced-learn
%pip install shap



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------------------------------------- -- 2.6/2.8 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   --- ------------------------------------ 2.6/30.3 MB 13.7 MB/s eta 0:00:03
   ------ --------------------------------- 5.2/30.3 MB 12.8 MB/s eta 0:00:02
   ---------- ----------------------------- 7.9/30.3 MB 12.5 MB/s eta 0:00:02
   ------------- -------------------------- 10.0/30.3 MB 11.9 MB/s eta 0:00:02
   ---------------- ----------------------- 12.8/30.3 MB 12.0 MB/s eta 0:00:02
   -------------------- ------------------- 15.5/30

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from imblearn.over_sampling import SMOTE
import shap


\\labdrive.tntech.edu\labdrive\rggrimaud42\Documents\GitHub\4260\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and split data

In [31]:
train_dir = "train_cleaned.csv"
train_data = pd.read_csv(train_dir)

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Columns: 107 entries, Available Extra Rooms in Hospital to City_Code_Patient_9
dtypes: bool(99), float64(7), int64(1)
memory usage: 49.5 MB


In [32]:
X = train_data.drop(columns=['Stay'])
X = X.astype('float32')
y = train_data['Stay']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)      

In [33]:
# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)


In [34]:
# Get the number of input features and classes
num_features = X_train_balanced.shape[1]
num_classes = len(np.unique(y_train_balanced))

In [35]:
print(X_train.dtypes)


Available Extra Rooms in Hospital    float32
Bed Grade                            float32
Type of Admission                    float32
Severity of Illness                  float32
Visitors with Patient                float32
                                      ...   
City_Code_Patient_5                  float32
City_Code_Patient_6                  float32
City_Code_Patient_7                  float32
City_Code_Patient_8                  float32
City_Code_Patient_9                  float32
Length: 106, dtype: object


## Neural Network

In [ ]:
# Build the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(num_features,)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if one-hot encoded
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_train, y_train), 
    epochs=20, 
    batch_size=32, 
    class_weight=None,  # Adjust if imbalance exists
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

\\labdrive.tntech.edu\labdrive\rggrimaud42\Documents\GitHub\4260\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
7961/7961 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.3069 - loss: 1.9016 - val_accuracy: 0.4009 - val_loss: 1.5699
Epoch 2/20
7961/7961 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.3874 - loss: 1.6154 - val_accuracy: 0.4139 - val_loss: 1.5481
Epoch 3/20
7961/7961 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.3969 - loss: 1.5939 - val_accuracy: 0.4159 - val_loss: 1.5385
Epoch 4/20
7961/7961 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4011 - loss: 1.5860 - val_accuracy: 0.4216 - val_loss: 1.5282
Epoch 5/20
7961/7961 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.4022 - loss: 1.5801 - val_accuracy: 0.4216 - val_loss: 1.5244
Epoch 6/20
7947/7961 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4038 - loss: 1.5764

In [ ]:
# 1. Create an explainer — for TensorFlow/Keras models, use DeepExplainer
explainer = shap.DeepExplainer(model, X_train[:100])  # use a small background set

# 2. Pick some samples to explain
X_explain = X_test[:10]  # choose a small number for visual clarity

# 3. Generate SHAP values
shap_values = explainer.shap_values(X_explain)

# 4. Plot!
shap.summary_plot(shap_values, X_explain)